In [30]:
import requests
import json
import sys
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain_openai import OpenAI
from langchain.document_loaders import TextLoader
from langchain.agents import AgentType, initialize_agent
from langchain.tools import Tool
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from langchain_community.utilities import WikipediaAPIWrapper

# url = 'http://127.0.0.1:1234/v1/chat/completions'
url = 'http://127.0.0.1:1234/v1'
headers = {"Content-Type": "application/json"}

model_1 = OpenAI(
    base_url = url,
    api_key = 'whatever',
    model_name = "llama-3.2-1b-instruct",
    temperature = 1.0
)

model_2 = OpenAI(
    base_url = url,
    api_key = 'whatever',
    model_name = "llama-3.2-1b-instruct",
    temperature = 0.0
)

model_3 = OpenAI(
    base_url = url,
    api_key = 'whatever',
    model_name = "llama-3.2-1b-instruct",
    temperature = 0.6
)

recommendation_prompt = PromptTemplate(
    input_variables=["interest"],
    template="Suggest a fascinating travel location described by text: {interest}. Output the name of strictly one location only. No descriptions or summary. Do not show thought process. Pure answers only."
)

recommendation_chain = LLMChain(llm=model_1, prompt=recommendation_prompt)

information_prompt = PromptTemplate(
    input_variables=["place"],
    template="Compile the most interesting facts about {place}. Focus on tourist attractions, cultural highlights, and practical information. Output purely the data. No more than 150 characters. Do not show thought process. Pure answers only."
)

information_chain = LLMChain(llm=model_2, prompt=information_prompt)

promoter_prompt = PromptTemplate(
    input_variables=["information"],
    template="You are a travel advisor. The following is information of a tourist destination: {information}. Write an enthusiastic promotion for this location in 5 sentences or less. Do not break character. Do not show thought process. Pure answers only. Do not ever acknowledge that you are responding to a prompt."
)

promoter_chain = LLMChain(llm=model_3, prompt=promoter_prompt)

# four_step_chain = SimpleSequentialChain(chains=[recommendation_chain, information_chain, promoter_chain])

search = DuckDuckGoSearchAPIWrapper()
wiki = WikipediaAPIWrapper()

tools = [
    Tool(
        name="DuckDuckGo Search",
        func=search.run,
        description="Useful for searching recent or general information on current events or locations."
    ),
    Tool(
        name="Wikipedia",
        func=wiki.run,
        description="Use this to get summaries and factual information about places, people, or concepts."
    )
]

def fetch_destination_info(destination):
    try:
        search_info = search.run(destination)
    except:
        return "No info found."

    if not search_info:
        return "No info found."

    return search_info
    
def get_response(user_input):
    place = recommendation_chain.run(user_input)
    
    information = information_chain.run(place=place)

    search_info = fetch_destination_info(place)

    combined_info = f"Search results: {search_info}\nLLM Summary: {information}"
    
    promotion = promoter_chain.run(combined_info)
    
    return promotion

def parse_input(user_input):
    match user_input:
        case 'Q': return None
        case _: return user_input

def run_llm():    
    while True:
        user_input = input()
        query = parse_input(user_input)
        
        if query == None:
            sys.exit()

        response = get_response(query)
        
        if response == None:
            input("Press Enter to try again...")
            continue
            
        print(response)

In [ ]:
run_llm()

 Beach, tropical, hot


 
Beach
- The beaches of Greece offer breathtaking views.
- The island of Borneo is famous for its orangutans.
- The term "paradise" was used by the ancient Greeks to describe Delos.

Cove
- Santorini's coves are stunning.
- Fiordland National Park has many beautiful coves.
- The beaches of Greece are perfect for summer vacations. 
- Borneo is home to a high concentration of orangutans and other wildlife. 

Beach
- Orangutans live in the wild.
- The term "paradise" refers to Delos.

Cove
- Santorini's coves have stunning views.
- Fiordland National Park has many beautiful coves.
- Borneo is a popular destination for hikers and kayakers. 
- Greece's beaches are perfect for relaxation. 

Beach
- The term "paradise" was used by the ancient Greeks to describe Delos.
- Orangutans live in the wild on Borneo.

Cove
- Santorini's coves have breathtaking views.
- Fiordland National Park has many beautiful coves.
- Greece's beaches are perfect for summer vacations. 
-
